In [ ]:
from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive
%cd /gdrive


Mounted at /gdrive
/gdrive


In [ ]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer 
from nltk.stem import LancasterStemmer 
from nltk.corpus import stopwords
nltk.download('stopwords')

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,classification_report
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore")



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#Read files
textfile = r'/gdrive/MyDrive/CIS508/Assignment_5/Problem_1_NLP/Comments.csv'
textData = pd.read_csv(textfile) #creates a dataframe

CustInfofile = r'/gdrive/MyDrive/CIS508/Assignment_5/Problem_1_NLP/Customers.csv'
CustInfoData = pd.read_csv(CustInfofile)  #creates a dataframe

print(textData.shape)
print(CustInfoData.shape)


(2070, 2)
(2070, 17)


In [ ]:
#Extract target column from Customer Info file
X = CustInfoData.drop(columns=["TARGET"]) #extracting training data without the target column
y = CustInfoData["TARGET"]
                     
print(X.shape)
print(textData.shape)
print(textData.head())
print(y)

(2070, 16)
(2070, 2)
     ID                                           Comments
0  1309  Does not like the way the phone works. It is t...
1  3556  Wanted to know the nearest store location. Wan...
2  2230  Wants to know how to do text messaging. Referr...
3  2312  Asked how to disable call waiting. referred hi...
4  3327  Needs help learning how to use the phone. I su...
0       Cancelled
1         Current
2         Current
3         Current
4       Cancelled
          ...    
2065    Cancelled
2066    Cancelled
2067    Cancelled
2068    Cancelled
2069    Cancelled
Name: TARGET, Length: 2070, dtype: object


Word Tokenizer

In [ ]:
#Tokenize - Split the sentences to lists of words
textData['CommentsTokenized'] = textData['Comments'].apply(word_tokenize)

In [ ]:
textData

,ID,Comments,CommentsTokenized
0,1309,Does not like the way the phone works. It is t...,"[Does, not, like, the, way, the, phone, works,..."
1,3556,Wanted to know the nearest store location. Wan...,"[Wanted, to, know, the, nearest, store, locati..."
2,2230,Wants to know how to do text messaging. Referr...,"[Wants, to, know, how, to, do, text, messaging..."
3,2312,Asked how to disable call waiting. referred hi...,"[Asked, how, to, disable, call, waiting, ., re..."
4,3327,Needs help learning how to use the phone. I su...,"[Needs, help, learning, how, to, use, the, pho..."
...,...,...,...
2065,3034,Needed help figuring out his bill. I explained...,"[Needed, help, figuring, out, his, bill, ., I,..."
2066,271,He lost his phone and called to cancel service...,"[He, lost, his, phone, and, called, to, cancel..."
2067,783,Lost the directions to phone and wants another...,"[Lost, the, directions, to, phone, and, wants,..."
2068,1295,Wants to change address.,"[Wants, to, change, address, .]"


Stemmer experience

In [ ]:
TD_snow=textData.copy()

In [ ]:
# Use English stemmer."Snowball"
stem_snow = SnowballStemmer("english")

#Now do stemming  
TD_snow['CommentsTokenizedStemmed'] = TD_snow['CommentsTokenized'].apply(lambda x: [stem_snow.stem(y) for y in x]) # Stem every word.
#Drop Unnecessary Columns 
TD_snow=TD_snow.drop(columns=["CommentsTokenized","Comments"])
#Save
#export_csv = TD_snow.to_csv(r'/gdrive/MyDrive/CIS508/Assignment_5/Problem_1_NLP/TD_snow.csv')


In [ ]:
TD_snow.head()

,ID,CommentsTokenizedStemmed
0,1309,"[doe, not, like, the, way, the, phone, work, ...."
1,3556,"[want, to, know, the, nearest, store, locat, ...."
2,2230,"[want, to, know, how, to, do, text, messag, .,..."
3,2312,"[ask, how, to, disabl, call, wait, ., refer, h..."
4,3327,"[need, help, learn, how, to, use, the, phone, ..."


In [ ]:
TD_lan=textData.copy()

In [ ]:
# Use English stemmer."Lancaster"
stem_lan = LancasterStemmer()

#Now do stemming - create a new dataframe to store stemmed version, 
TD_lan['CommentsTokenizedStemmed'] = TD_lan['CommentsTokenized'].apply(lambda x: [stem_lan.stem(y) for y in x]) 
#Drop Unnecessary Columns 
TD_lan=TD_lan.drop(columns=["CommentsTokenized","Comments"])
#Save
#export_csv = TD_lan.to_csv(r'/gdrive/MyDrive/CIS508/Assignment_5/Problem_1_NLP/TD_lan.csv')

In [ ]:
TD_lan.head()

,ID,CommentsTokenizedStemmed
0,1309,"[doe, not, lik, the, way, the, phon, work, ., ..."
1,3556,"[want, to, know, the, nearest, stor, loc, ., w..."
2,2230,"[want, to, know, how, to, do, text, mess, ., r..."
3,2312,"[ask, how, to, dis, cal, wait, ., refer, him, ..."
4,3327,"[nee, help, learn, how, to, us, the, phon, ., ..."


In [ ]:
print(stem_lan.stem('phone'))
print(stem_snow.stem('phone'))

phon
phone


In [ ]:
#Join stemmed strings : Snow Stemmer
TD_snow['CommentsTokenizedStemmed'] = TD_snow['CommentsTokenizedStemmed'].apply(lambda x: " ".join(x))

#export_csv = TD_Snow.to_csv(r'/gdrive/MyDrive/CIS508/Assignment_5/Problem_1_NLP/TD-Joined.csv')

Stop word 

In [ ]:
#Stop word 
count_vect = CountVectorizer(stop_words='english',lowercase=False)
TD_counts = count_vect.fit_transform(TD_snow.CommentsTokenizedStemmed)
print(TD_counts.shape)
print(TD_counts.dtype)
print(count_vect.get_feature_names())

(2070, 354)
int64
['3399', '3g', 'abysm', 'access', 'accessori', 'adapt', 'add', 'addit', 'additon', 'address', 'adit', 'adress', 'advertis', 'afraid', 'alway', 'angel', 'angri', 'ani', 'anoth', 'anyth', 'anytim', 'area', 'asap', 'ask', 'bad', 'basic', 'bateri', 'batteri', 'becaus', 'believ', 'better', 'bigger', 'book', 'bought', 'brain', 'bring', 'built', 'busi', 'button', 'buy', 'cancel', 'cancer', 'car', 'care', 'carrier', 'caus', 'cc', 'cell', 'certain', 'chang', 'charg', 'charger', 'check', 'chip', 'citi', 'claim', 'cleariti', 'cold', 'comapr', 'compani', 'compar', 'competit', 'complain', 'complaint', 'concept', 'connect', 'consisit', 'consist', 'constan', 'contact', 'continu', 'contract', 'correct', 'cost', 'coupl', 'cover', 'coverag', 'creat', 'credit', 'cstmer', 'cstmr', 'current', 'cust', 'custom', 'customr', 'date', 'day', 'dead', 'decent', 'defect', 'deo', 'did', 'die', 'differ', 'difficult', 'digiti', 'direct', 'disabl', 'doe', 'don', 'dont', 'drop', 'dure', 'easier', 'effe

Term Document Matrix

In [ ]:
#Change to DataFrame
DF_TD_Counts=pd.DataFrame(TD_counts.toarray())
print(DF_TD_Counts)
export_csv = DF_TD_Counts.to_csv(r'/gdrive/MyDrive/CIS508/Assignment_5/Problem_1_NLP/TD_counts-TokenizedStemmed.csv')


      0    1    2    3    4    5    6    ...  347  348  349  350  351  352  353
0       0    0    0    0    0    0    0  ...    1    0    0    0    0    0    0
1       0    0    0    0    1    0    0  ...    0    0    0    0    0    0    0
2       0    0    0    0    0    0    0  ...    0    0    0    0    0    0    0
3       0    0    0    0    0    0    0  ...    0    0    0    0    0    0    0
4       0    0    0    0    0    0    0  ...    0    0    0    0    0    0    0
...   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...
2065    0    0    0    0    0    0    0  ...    0    0    0    0    0    0    0
2066    0    0    0    0    0    0    0  ...    0    0    0    0    0    0    0
2067    0    0    0    0    0    0    0  ...    0    0    0    0    0    0    0
2068    0    0    0    0    0    0    0  ...    0    0    0    0    0    0    0
2069    0    0    0    0    0    0    0  ...    0    0    0    0    0    0    0

[2070 rows x 354 columns]


TF-IDF

In [ ]:
#Compute TF-IDF Matrix
tfidf_transformer = TfidfTransformer()
X_tfidf = tfidf_transformer.fit_transform(TD_counts)
print(X_tfidf.shape)
DF_TF_IDF=pd.DataFrame(X_tfidf.toarray())
print(DF_TF_IDF)
export_csv= DF_TF_IDF.to_csv(r'/gdrive/MyDrive/CIS508/Assignment_5/Problem_1_NLP/TFIDF_counts-TokenizedStemmed.csv')


(2070, 354)
      0    1    2    3        4    5    ...  348  349  350  351  352  353
0     0.0  0.0  0.0  0.0  0.00000  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
1     0.0  0.0  0.0  0.0  0.27568  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
2     0.0  0.0  0.0  0.0  0.00000  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
3     0.0  0.0  0.0  0.0  0.00000  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
4     0.0  0.0  0.0  0.0  0.00000  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
...   ...  ...  ...  ...      ...  ...  ...  ...  ...  ...  ...  ...  ...
2065  0.0  0.0  0.0  0.0  0.00000  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
2066  0.0  0.0  0.0  0.0  0.00000  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
2067  0.0  0.0  0.0  0.0  0.00000  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
2068  0.0  0.0  0.0  0.0  0.00000  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
2069  0.0  0.0  0.0  0.0  0.00000  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0

[2070 rows x 354 columns]


Combine Customer Data

In [ ]:
#Merge files
combined=pd.concat([X, DF_TF_IDF], axis=1)

print(CustInfoData.shape)
print(X.shape)
print(combined.shape)
print(combined)
export_csv= combined.to_csv(r'/gdrive/MyDrive/CIS508/Assignment_5/Problem_1_NLP/Combined-Cust+TFIDF+SelectedFeatures.csv')

(2070, 17)
(2070, 16)
(2070, 370)
        ID Sex Status  Children  Est_Income  ...  349  350  351  352  353
0        1   F      S         1    38000.00  ...  0.0  0.0  0.0  0.0  0.0
1        6   M      M         2    29616.00  ...  0.0  0.0  0.0  0.0  0.0
2        8   M      M         0    19732.80  ...  0.0  0.0  0.0  0.0  0.0
3       11   M      S         2       96.33  ...  0.0  0.0  0.0  0.0  0.0
4       14   F      M         2    52004.80  ...  0.0  0.0  0.0  0.0  0.0
...    ...  ..    ...       ...         ...  ...  ...  ...  ...  ...  ...
2065  3821   F      S         0    78851.30  ...  0.0  0.0  0.0  0.0  0.0
2066  3822   F      S         1    17540.70  ...  0.0  0.0  0.0  0.0  0.0
2067  3823   F      M         0    83891.90  ...  0.0  0.0  0.0  0.0  0.0
2068  3824   F      M         2    28220.80  ...  0.0  0.0  0.0  0.0  0.0
2069  3825   F      S         0    28589.10  ...  0.0  0.0  0.0  0.0  0.0

[2070 rows x 370 columns]


One Hot encording

In [ ]:
#Do one Hot encoding for categorical features
X_cat = ["Sex","Status","Car_Owner","Paymethod","LocalBilltype","LongDistanceBilltype"]
combined_train = pd.get_dummies(combined,columns=X_cat)
print(combined_train)

        ID  ...  LongDistanceBilltype_Standard
0        1  ...                              0
1        6  ...                              1
2        8  ...                              1
3       11  ...                              1
4       14  ...                              0
...    ...  ...                            ...
2065  3821  ...                              1
2066  3822  ...                              1
2067  3823  ...                              1
2068  3824  ...                              1
2069  3825  ...                              1

[2070 rows x 378 columns]


In [ ]:
combined_train.head()

,ID,Children,Est_Income,Usage,Age,RatePlan,LongDistance,International,Local,Dropped,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,...,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,Sex_F,Sex_M,Status_D,Status_M,Status_S,Car_Owner_N,Car_Owner_Y,Paymethod_Auto,Paymethod_CC,Paymethod_CH,LocalBilltype_Budget,LocalBilltype_FreeLocal,LongDistanceBilltype_Intnl_discount,LongDistanceBilltype_Standard
0,1,1,38000.00,229.64,24.393333,3,23.56,0.0,206.08,0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.388928,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.209678,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,1,1,0,0,1,0,1,0,1,0
1,6,2,29616.00,75.29,49.426667,2,29.78,0.0,45.50,0,0.0,0.0,0.0,0.0,0.27568,0.0,0.0,0.0,0.0,0.0,0.708478,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.312065,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,1,0,1,0,0,0,1,0,1,0,1
2,8,0,19732.80,47.25,50.673333,3,24.81,0.0,22.44,0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.195324,0.0,0.000000,0.0,0.000000,0.567289,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,1,0,1,0,0,1,0,0,1,0,1
3,11,2,96.33,59.01,56.473333,1,26.13,0.0,32.88,1,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.352868,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.480064,0.000000,0.0,0.000000,0.0,0.333825,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,1,1,0,0,1,0,1,0,0,1
4,14,2,52004.80,28.14,25.140000,1,5.03,0.0,23.11,0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.348322,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1,0,1,0,0,0,1,1,0,1,0


Feature Selection

1. Filter Training

In [ ]:
trainset=combined_train.copy()

In [ ]:
#Suppose, we select 50 features with top  Fisher scores
selector = SelectKBest(k=50)
combined_train_K50 = selector.fit_transform(trainset,y) 
feature_names_out = selector.get_support(indices=True) 
combined_train_K50_SelectedFeatures= pd.DataFrame(combined_train_K50)

print("=== Top 50 Features ===")
print(combined_train_K50_SelectedFeatures)
combined_train_K50.shape


#Suppose, we select 100 features with top Fisher scores
trainset=combined_train.copy()
selector = SelectKBest(k=100)
combined_train_K100 = selector.fit_transform(trainset,y) 
feature_names_out = selector.get_support(indices=True) 
combined_train_K100_SelectedFeatures= pd.DataFrame(combined_train_K100)

print("=== Top 100 Features ===")
print(combined_train_K100_SelectedFeatures)
combined_train_K100.shape


#Suppose, we select 150 features with top Fisher scores
trainset=combined_train.copy()
selector = SelectKBest(k=150)
combined_train_K150 = selector.fit_transform(trainset,y)
feature_names_out = selector.get_support(indices=True)
combined_train_K150_SelectedFeatures= pd.DataFrame(combined_train_K150)

print("=== Top 150 Features ===")
print(combined_train_K150_SelectedFeatures)
combined_train_K150.shape

#Suppose, we select 200 features with top Fisher scores
trainset=combined_train.copy()
selector = SelectKBest(k=200)
combined_train_K200 = selector.fit_transform(trainset,y)
feature_names_out = selector.get_support(indices=True) 
combined_train_K200_SelectedFeatures= pd.DataFrame(combined_train_K200)

print("=== Top 200 Features ===")
print(combined_train_K200_SelectedFeatures)
combined_train_K200.shape

=== Top 50 Features ===
       0         1       2      3     4       5   ...   44   45   46   47   48   49
0     1.0  38000.00  229.64  23.56  0.00  206.08  ...  0.0  1.0  0.0  1.0  1.0  0.0
1     2.0  29616.00   75.29  29.78  0.00   45.50  ...  1.0  0.0  0.0  0.0  0.0  1.0
2     0.0  19732.80   47.25  24.81  0.00   22.44  ...  1.0  0.0  0.0  1.0  0.0  1.0
3     2.0     96.33   59.01  26.13  0.00   32.88  ...  0.0  1.0  0.0  1.0  0.0  1.0
4     2.0  52004.80   28.14   5.03  0.00   23.11  ...  1.0  0.0  0.0  0.0  1.0  0.0
...   ...       ...     ...    ...   ...     ...  ...  ...  ...  ...  ...  ...  ...
2065  0.0  78851.30   29.04   0.37  0.00   28.66  ...  0.0  1.0  0.0  1.0  0.0  1.0
2066  1.0  17540.70   36.20  22.17  0.57   13.45  ...  0.0  1.0  1.0  0.0  0.0  1.0
2067  0.0  83891.90   74.40  28.92  0.00   45.47  ...  1.0  0.0  0.0  0.0  0.0  1.0
2068  2.0  28220.80   38.95  26.49  0.00   12.46  ...  1.0  0.0  0.0  1.0  0.0  1.0
2069  0.0  28589.10  100.28  13.19  0.00   87.09  ..

(2070, 200)

In [ ]:
combined_train.shape
print(feature_names_out)


[  1   2   3   4   5   6   7   8   9  10  12  13  14  16  17  19  20  21
  24  26  27  29  30  33  34  35  36  37  39  40  41  45  46  49  53  54
  56  57  58  59  60  61  62  65  66  67  70  71  72  74  76  77  80  82
  83  88  89  90  91  92  93  94  97  98 101 102 104 105 109 116 119 124
 125 128 130 131 135 136 138 140 141 142 143 145 147 148 149 151 154 155
 158 159 161 164 166 167 168 170 171 174 176 177 178 180 182 183 184 185
 186 192 193 196 200 203 204 207 208 211 214 215 216 218 219 221 222 223
 226 227 230 231 232 234 238 239 240 244 245 248 249 254 255 258 259 264
 266 267 268 269 270 271 272 274 276 277 282 283 287 288 290 293 296 297
 298 299 305 306 312 317 322 323 325 328 329 330 331 333 335 337 338 341
 342 343 346 348 350 351 352 354 358 360 361 362 364 365 367 368 371 372
 376 377]


In [ ]:
#Construct classifiers to test candidate feature selection subgroups
#Gradient boosting Classifier


gb_K50= GradientBoostingClassifier().fit(combined_train_K50_SelectedFeatures,y)
clf_cv_score = cross_val_score(gb_K50, combined_train_K50_SelectedFeatures,y, cv=20, scoring="accuracy")

print("=== All Accuracy Scores ===")
print(clf_cv_score)
print('\n')
print("=== Mean Accuracy Score ===")
print("Mean Accuracy Score - ON Text: ",clf_cv_score.mean())
print('\n')


gb_K100= GradientBoostingClassifier().fit(combined_train_K100_SelectedFeatures, y)
clf_cv_score = cross_val_score(gb_K100, combined_train_K100_SelectedFeatures, y, cv=20, scoring="accuracy")

print("=== All Accuracy Scores ===")
print(clf_cv_score)
print('\n')
print("=== Mean Accuracy Score ===")
print("Mean Accuracy Score - ON Text: ",clf_cv_score.mean())
print('\n')

gb_K150= GradientBoostingClassifier().fit(combined_train_K150_SelectedFeatures, y)
clf_cv_score = cross_val_score(gb_K150, combined_train_K150_SelectedFeatures, y, cv=20, scoring="accuracy")

print("=== All Accuracy Scores ===")
print(clf_cv_score)
print('\n')
print("=== Mean Accuracy Score ===")
print("Mean Accuracy Score - ON Text: ",clf_cv_score.mean())
print('\n')


gb_K200= GradientBoostingClassifier().fit(combined_train_K200_SelectedFeatures, y)
clf_cv_score = cross_val_score(gb_K150, combined_train_K200_SelectedFeatures, y, cv=20, scoring="accuracy")

print("=== All Accuracy Scores ===")
print(clf_cv_score)
print('\n')
print("=== Mean Accuracy Score ===")
print("Mean Accuracy Score - ON Text: ",clf_cv_score.mean())
print('\n')



=== All Accuracy Scores ===
[0.84615385 0.84615385 0.81730769 0.875      0.85576923 0.85576923
 0.88461538 0.77884615 0.81730769 0.90384615 0.90291262 0.87378641
 0.94174757 0.81553398 0.86407767 0.84466019 0.91262136 0.9223301
 0.83495146 0.86407767]


=== Mean Accuracy Score ===
Mean Accuracy Score - ON Text:  0.8628734129947722


=== All Accuracy Scores ===
[0.85576923 0.84615385 0.81730769 0.875      0.83653846 0.86538462
 0.875      0.79807692 0.82692308 0.89423077 0.91262136 0.88349515
 0.94174757 0.7961165  0.88349515 0.82524272 0.90291262 0.9223301
 0.83495146 0.85436893]


=== Mean Accuracy Score ===
Mean Accuracy Score - ON Text:  0.8623833084391338


=== All Accuracy Scores ===
[0.85576923 0.85576923 0.82692308 0.875      0.84615385 0.83653846
 0.88461538 0.76923077 0.84615385 0.90384615 0.9223301  0.90291262
 0.94174757 0.83495146 0.88349515 0.84466019 0.9223301  0.91262136
 0.84466019 0.89320388]


=== Mean Accuracy Score ===
Mean Accuracy Score - ON Text:  0.8701456310679

(2) Warpper Training

  Model (1) Random Forest

In [ ]:
trainset_w=combined_train.copy()

In [ ]:
rfc = RandomForestClassifier()
rfc.fit(trainset_w, y)

model = SelectFromModel(rfc, prefit=True)

X_warp_rfc = model.transform(trainset_w)
X_warp_rfc_SelectedFeatures= pd.DataFrame(X_warp_rfc)

print(X_warp_rfc_SelectedFeatures)

cv_score = cross_val_score(rfc, X_warp_rfc_SelectedFeatures, y, cv=20, scoring="accuracy")
print("=== All Accuracy Scores(SelectFromModel) ===")
print(cv_score)
print('\n')
print("=== Mean Accuracy Score(SelectFromModel) ===")
print("Mean Accuracy Score - ON Text: ",cv_score.mean())
print('\n')
#export_csv= X_new_SelectedFeatures.to_csv(r'/gdrive/My Drive/CIS508-FALL2020/PT5B/X_new_SelectedFeatures.csv')

          0    1         2       3          4   ...   45   46   47   48   49
0        1.0  1.0  38000.00  229.64  24.393333  ...  0.0  1.0  0.0  1.0  0.0
1        6.0  2.0  29616.00   75.29  49.426667  ...  1.0  0.0  1.0  0.0  1.0
2        8.0  0.0  19732.80   47.25  50.673333  ...  0.0  0.0  1.0  0.0  1.0
3       11.0  2.0     96.33   59.01  56.473333  ...  0.0  1.0  0.0  0.0  1.0
4       14.0  2.0  52004.80   28.14  25.140000  ...  1.0  1.0  0.0  1.0  0.0
...      ...  ...       ...     ...        ...  ...  ...  ...  ...  ...  ...
2065  3821.0  0.0  78851.30   29.04  48.373333  ...  0.0  0.0  1.0  0.0  1.0
2066  3822.0  1.0  17540.70   36.20  62.786667  ...  0.0  1.0  0.0  0.0  1.0
2067  3823.0  0.0  83891.90   74.40  61.020000  ...  1.0  1.0  0.0  0.0  1.0
2068  3824.0  2.0  28220.80   38.95  38.766667  ...  0.0  0.0  1.0  0.0  1.0
2069  3825.0  0.0  28589.10  100.28  15.600000  ...  0.0  0.0  1.0  0.0  1.0

[2070 rows x 50 columns]
=== All Accuracy Scores(SelectFromModel) ===
[0.78

Model (2) Gradient Boost

In [ ]:
trainset_w2=combined_train.copy()

In [ ]:
#Feature Selection (Wrapper method)

gb = GradientBoostingClassifier()
gb.fit(trainset_w2, y)

model = SelectFromModel(gb, prefit=True)

X_warp_gb = model.transform(trainset_w2)
X_warp_gb_SelectedFeatures= pd.DataFrame(X_warp_gb)

print(X_warp_gb_SelectedFeatures)

cv_score = cross_val_score(gb, X_warp_gb_SelectedFeatures, y, cv=20, scoring="accuracy")
print("=== All Accuracy Scores(SelectFromModel) ===")
print(cv_score)
print('\n')
print("=== Mean Accuracy Score(SelectFromModel) ===")
print("Mean Accuracy Score - ON Text: ",cv_score.mean())
print('\n')

          0    1         2       3          4   ...   12   13   14   15   16
0        1.0  1.0  38000.00  229.64  24.393333  ...  0.0  1.0  0.0  0.0  0.0
1        6.0  2.0  29616.00   75.29  49.426667  ...  1.0  0.0  0.0  1.0  1.0
2        8.0  0.0  19732.80   47.25  50.673333  ...  1.0  0.0  0.0  0.0  1.0
3       11.0  2.0     96.33   59.01  56.473333  ...  0.0  1.0  0.0  0.0  1.0
4       14.0  2.0  52004.80   28.14  25.140000  ...  1.0  0.0  0.0  1.0  0.0
...      ...  ...       ...     ...        ...  ...  ...  ...  ...  ...  ...
2065  3821.0  0.0  78851.30   29.04  48.373333  ...  0.0  1.0  0.0  0.0  1.0
2066  3822.0  1.0  17540.70   36.20  62.786667  ...  0.0  1.0  1.0  0.0  1.0
2067  3823.0  0.0  83891.90   74.40  61.020000  ...  1.0  0.0  0.0  1.0  1.0
2068  3824.0  2.0  28220.80   38.95  38.766667  ...  1.0  0.0  0.0  0.0  1.0
2069  3825.0  0.0  28589.10  100.28  15.600000  ...  0.0  1.0  0.0  0.0  1.0

[2070 rows x 17 columns]
=== All Accuracy Scores(SelectFromModel) ===
[0.74

In [ ]:
#Train Test Split 8:2
X_train, X_test, y_train, y_test = train_test_split(
combined_train_K200_SelectedFeatures, y, test_size=0.2, random_state=17)

In [ ]:
#Filter Method Best features
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
pred = rfc.predict(X_test)
print(accuracy_score(y_test, pred))

0.8768115942028986


In [ ]:
#Train Test Split 8:2
X_train, X_test, y_train, y_test = train_test_split(
X_warp_rfc_SelectedFeatures, y, test_size=0.2, random_state=17)

In [ ]:
#Wrapper Method Best features
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
pred = rfc.predict(X_test)
print(accuracy_score(y_test, pred))

0.8888888888888888
